# JOINs

In [1]:
USE NormalisierungsDemo

Commands completed successfully.

Total execution time: 00:00:00.001

Zeigen Sie Abteilung und Die Personen, Die in den Abteilungen Arbeiten

In [9]:
SELECT a.Name As Abteilung, p.Vorname AS [Mitarbeiterer Vorname], p.Name As [ Mitarbeiterer Nachname]
FROM Personal AS p
JOIN Abteilung AS a ON a.AbtNr=p.AbtNr

(4 rows affected)

Total execution time: 00:00:00.039

Abteilung,Mitarbeiterer Vorname,Mitarbeiterer Nachname
Personal,Maria,Eichenau
Einkauf,Stefanie,Glahn
Einkauf,Karin,Kirsch
Verkauf,Sean,Conolly


Zeigen Sie die Projekte , die im Projekt tätigen Personen, die ausgeübte Tätigkeit und Die Anzahl Stunden

In [3]:
SELECT p.Beschreibung AS Projekt, pe.Vorname, pe.Name, t.Tätigkeit
FROM Projekt AS p 
JOIN Arbeitet_An As aa ON aa.ProjNr=p.ProjNr
JOIN Personal AS pe ON pe.PersonalNr=aa.PersonalNr
JOIN Tätigkeit AS t ON t.TätigkeitsNr=aa.TätigkeitsNr

(7 rows affected)

Total execution time: 00:00:00.074

Beschreibung,Vorname,Name,Tätigkeit
Kundenumfrage,Stefanie,Glahn,Leitung
Konkurrenzanalyse,Stefanie,Glahn,Bearbeitung
Kundenumfrage,Karin,Kirsch,Bearbeitung
Konkurrenzanalyse,Karin,Kirsch,Bearbeitung
Kundenumfrage,Sean,Conolly,Bearbeitung
Verkaufsmesse,Sean,Conolly,Leitung
Konkurrenzanalyse,Sean,Conolly,Leitung


In [ ]:
Geben Sie zusätzlich das den geleisteten Stunden entsprechende Honorar aus

In [10]:
SELECT p.Beschreibung, pe.Vorname AS [Mitarbeiterer Vorname], pe.Name AS [Mitarbeiterer Nachname], t.Tätigkeit , aa.Stunden*t.Stundenlohn As Honorar
FROM Projekt AS p 
JOIN Arbeitet_An As aa ON aa.ProjNr=p.ProjNr
JOIN Personal AS pe ON pe.PersonalNr=aa.PersonalNr
JOIN Tätigkeit AS t ON t.TätigkeitsNr=aa.TätigkeitsNr

(7 rows affected)

Total execution time: 00:00:00.044

Beschreibung,Mitarbeiterer Vorname,Mitarbeiterer Nachname,Tätigkeit,Honorar
Kundenumfrage,Stefanie,Glahn,Leitung,1250.00
Konkurrenzanalyse,Stefanie,Glahn,Bearbeitung,2400.00
Kundenumfrage,Karin,Kirsch,Bearbeitung,1650.00
Konkurrenzanalyse,Karin,Kirsch,Bearbeitung,1950.00
Kundenumfrage,Sean,Conolly,Bearbeitung,2100.00
Verkaufsmesse,Sean,Conolly,Leitung,1250.00
Konkurrenzanalyse,Sean,Conolly,Leitung,1250.00


# Transaktionen

Mustafa Özdemir ist neuer Mitarbeiter in Vekuaf. PersonalNR6   er arbeitet im Projekt Konkurrrenz analyse 20 Stunden an der Präsentätionvorbereitung

In [11]:
SELECT * FROM Tätigkeit

(3 rows affected)

Total execution time: 00:00:00.057

TätigkeitsNr,Tätigkeit,Stundenlohn
1,Leitung,50.00
2,Bearbeitung,30.00
3,Präsentationsvorbereitung,35.00


In [5]:

BEGIN
    BEGIN TRANSACTION;
    BEGIN TRY
        DECLARE @AbtNr INT, @ProjNr INT, @TätigkeitsNr INT;

        SET @AbtNr = (
            SELECT a.AbtNr
            FROM Abteilung AS a
            WHERE a.Name = 'Verkauf'
        );

        IF @AbtNr IS NULL
            THROW 50000, 'Abteilung nicht gefunden', 1;

        SET @ProjNr = (
            SELECT p.ProjNr
            FROM Projekt AS p
            WHERE p.Beschreibung = 'Konkurrenzanalyse'
        );

        IF @ProjNr IS NULL
            THROW 50000, 'Projekt nicht gefunden', 1;

        SET @TätigkeitsNr = (
            SELECT t.TätigkeitsNr
            FROM Tätigkeit AS t 
            WHERE t.Tätigkeit = 'Präsentationsvorbereitung'
        );

        IF @TätigkeitsNr IS NULL
            THROW 50000, 'Tätigkeit nicht gefunden', 1;

            

        -- jetz darf ich schreiben :-)
        /*INSERT INTO Personal (PersonalNr, Vorname, Name, AbtNr)
        VALUES (6,'Hans Georg','Röder',(
        SELECT a.AbtNr
        FROM Abteilung AS a
        WHERE a.Name = 'Verkauf'
    
    ))*/
        INSERT INTO Personal (PersonalNr, Vorname, Name, AbtNr)
        VALUES (6, 'Hans Georg', 'Röder', @AbtNr);

        INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
        VALUES (6, @ProjNr, @TätigkeitsNr, 20);

        COMMIT TRANSACTION;
    END TRY 
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH;
END


(0 rows affected)

: Msg 2627, Level 14, State 1, Line 44
Verletzung der PRIMARY KEY-Einschränkung "pk_Personal". Ein doppelter Schlüssel kann in das dbo.Personal-Objekt nicht eingefügt werden. Der doppelte Schlüsselwert ist (6).

Total execution time: 00:00:00.025

In [7]:
SELECT * FROM Personal;

(5 rows affected)

Total execution time: 00:00:00.048

PersonalNr,Name,Vorname,AbtNr
1,Eichenau,Maria,1
2,Glahn,Stefanie,2
3,Kirsch,Karin,2
4,Conolly,Sean,3
6,Röder,Hans Georg,3


In [15]:
SELECT a.AbtNr
FROM Abteilung AS a
WHERE a.Name = 'Verkauf'

(1 row affected)

Total execution time: 00:00:00.012

AbtNr
3


In [14]:
SELECT p.ProjNr
FROM Projekt AS p
WHERE p.Beschreibung='Konkurrenzanalyse';

(1 row affected)

Total execution time: 00:00:00.018

ProjNr
3


In [16]:
SELECT t.Tätigkeit
FROM Tätigkeit AS t 
WHERE t.Tätigkeit='Präsentationsvorbereitung'

(1 row affected)

Total execution time: 00:00:00.022

Tätigkeit
Präsentationsvorbereitung
